In [2]:
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import particles

from scipy.linalg import block_diag
from particles.collectors import Moments
from particles import state_space_models as ssm
from particles import distributions as dists

In [3]:
# to be removed
import black
import jupyter_black

jupyter_black.load(
    lab=False,
    line_length=119
)

<IPython.core.display.Javascript object>

# Two leg motion model
We consider a motion model, consisting of four rigid parts (left fibula, left femur, right fibula, right femur) and three hinges (left knee, hip, right knee). For simplicity, we assume the motion happens in the saggital plane, i.e. the motion is 2-dimensional:
<img src='images/TwoLegModel.png' width="500" height="500">

### States
We use the following states (blue), which describe the motion of this two leg model:

- position of the hip $ (x_H, y_H) $
- angle between vertical and left femur $ \varphi_0 $
- angle between left femur and left fibula $ \varphi_1 $
- angle between vertical and right femur $ \varphi_2 $
- angle between right femur and right fibula $ \varphi_3 $
- first and second derivatives of the above quantities: $ \dot x_H, \dot y_H, \ldots , \ddot \varphi_2, \ddot \varphi_3 $

In total, the state space is 36-dimensional $ x_t = (x_H, y_H, \varphi_0, \ldots \varphi_2, \ddot \varphi_3)^{T} \in \mathbb{R}^{36} $.

### Observations
As observations (orange), we use four inertial measurement units (IMU) and two pressure sensors at the feet. Each IMU measures linear accelerations $ \ddot x $- and $  \ddot y $ and angular velocity $ \omega $ in the $ x $-$ y $-plane:
$$
y^{\text{IMU}_{i}} = (\ddot x^{(i)}, \ddot y^{(i)}, \dot \omega^{(i)}), \quad i=0, 1, 2, 3
$$
The two pressure sensors only detect if there is ground contact or not. We can compute the expected velocity and acceleration from the states. If a sensor signals ground contact, we expect our prediction for velocity and acceleration to be close to 0. Another implication is that there will be times, when we have no signal from a pressure sensor, i.e. missing observations.
For each feet, we have four observation variables:
$$
y^{\text{PS}_{i}} = (\dot x^{(i)}, \dot y^{(i)}, \ddot x^{(i)}, \ddot y^{(i)}), \quad i=4, 5
$$
In total, the observation space is $ 4 \cdot 3 + 2 \cdot 8 = 20 $-dimensional $ y_{t} = (\ddot x^{(0)}, \ddot y^{(0)}, \dot \omega^{(0)}, \ldots , \dot x^{(5)}, \dot y^{(5)}, \ddot x^{(5)}, \ddot y^{(5)})^T \in \mathbb{R}^{20} $

In [27]:
states_truth = pd.read_csv("data/normal/states.csv")
observations = pd.read_csv("data/normal/observations.csv")

In [28]:
states_truth.head()

,$x_H$,$y_H$,$\varphi_0$,$\varphi_1$,$\varphi_2$,$\varphi_3$,$\dot x_H$,$\dot y_H$,$\dot \varphi_0$,$\dot \varphi_1$,$\dot \varphi_2$,$\dot \varphi_3$,$\ddot x_H$,$\ddot y_H$,$\ddot \varphi_0$,$\ddot \varphi_1$,$\ddot \varphi_2$,$\ddot \varphi_3 $
0,0.005679,1.057527,-0.128463,-0.247931,0.366397,-0.189801,0.567903,0.096320,2.536291,-3.798610,-0.078163,-0.818193,-4.070523e-11,0.005052,-1.776230,3.315853,-0.295288,0.535814
1,0.011358,1.058492,-0.103236,-0.285663,0.365574,-0.197908,0.567903,0.096974,2.504285,-3.738704,-0.089268,-0.798176,-1.251775e-10,0.128222,-4.654972,8.721949,-1.955715,3.521541
2,0.017037,1.059471,-0.078476,-0.322520,0.364548,-0.205651,0.567903,0.099096,2.442664,-3.623065,-0.119473,-0.743827,-2.756526e-10,0.298159,-7.674005,14.416044,-4.105452,7.384565
3,0.022716,1.060479,-0.054470,-0.357959,0.363123,-0.212674,0.567903,0.102600,2.354972,-3.458162,-0.167874,-0.656751,-5.076412e-10,0.397818,-9.805368,18.454107,-5.525057,9.941562
4,0.028395,1.061526,-0.031434,-0.391573,0.361152,-0.218716,0.567903,0.106852,2.250174,-3.260731,-0.227607,-0.549239,-8.333688e-10,0.448776,-11.084394,20.902107,-6.375846,11.479190


In [29]:
observations.head()

,$\ddot x^0$,$\ddot y^0$,$\omega_z^0$,$\ddot x^1$,$\ddot y^1$,$\omega_z^1$,$\ddot x^2$,$\ddot y^2$,$\omega_z^2$,$\ddot x^3$,$\ddot y^3$,$\omega_z^3$,$\dot x^4$,$\dot y^4$,$\ddot x^4$,$\ddot y^4$,$\dot x^5$,$\dot y^5$,$\ddot x^5$,$\ddot y^5$
0,-1.821814,11.900176,2.550071,-4.852858,12.373303,-1.280894,3.385255,8.980552,-0.080017,1.492541,9.627988,-0.875733,1.121228,0.212252,0.741781,3.858301,0.001966,-0.012164,-0.081558,0.454939
1,-2.361384,11.853904,2.514129,-6.011847,11.825068,-1.240908,3.043565,8.880006,-0.092579,1.060646,9.790665,-0.871481,1.128034,0.248770,0.612685,3.407860,0.001219,-0.007841,-0.067200,0.405066
2,-2.962156,11.609948,2.441278,-6.424099,11.467285,-1.204667,2.108684,9.413049,-0.110925,0.337125,9.618804,-0.888815,1.133419,0.279801,0.459334,2.763782,0.000638,-0.004161,-0.048449,0.325825
3,-4.042005,11.968531,2.381178,-6.927154,10.534151,-1.080061,2.099710,9.573533,-0.156228,0.342423,9.868236,-0.789812,1.137230,0.303813,0.302377,2.022525,0.000253,-0.001372,-0.028578,0.230321
4,-3.883105,11.748364,2.242771,-7.394055,9.943406,-1.029109,1.617350,9.982116,-0.225111,-0.265758,9.721602,-0.793610,1.139544,0.320394,0.163058,1.293955,0.000054,0.000473,-0.011828,0.139640


In [6]:
def create_rotation_matrix_z(alpha):
    R = np.array([[np.cos(alpha), np.sin(alpha), 0.0], [-np.sin(alpha), np.cos(alpha), 0.0], [0.0, 0.0, 1.0]])
    return R


def state_to_obs(x, dim_observations, g, legs, imus, R):
    nb_steps, _ = x.shape
    y = np.empty((nb_steps, dim_observations))
    # left femur
    y[:, 0] = imus[0] * x[:, 14] + g * np.sin(x[:, 2]) + np.sin(x[:, 2]) * x[:, 13] + np.cos(x[:, 2]) * x[:, 12]
    y[:, 1] = imus[0] * x[:, 8] ** 2 + g * np.cos(x[:, 2]) - np.sin(x[:, 2]) * x[:, 12] + np.cos(x[:, 2]) * x[:, 13]
    y[:, 2] = x[:, 8]

    # left fibula
    y[:, 3] = (
        imus[1] * x[:, 14]
        + imus[1] * x[:, 15]
        + g * np.sin(x[:, 2] + x[:, 3])
        + legs[0] * np.sin(x[:, 3]) * x[:, 8] ** 2
        + legs[0] * np.cos(x[:, 3]) * x[:, 14]
        + np.sin(x[:, 2] + x[:, 3]) * x[:, 13]
        + np.cos(x[:, 2] + x[:, 3]) * x[:, 12]
    )
    y[:, 4] = (
        imus[1] * x[:, 8] ** 2
        + 2 * imus[1] * x[:, 8] * x[:, 9]
        + imus[1] * x[:, 9] ** 2
        + g * np.cos(x[:, 2] + x[:, 3])
        - legs[0] * np.sin(x[:, 3]) * x[:, 14]
        + legs[0] * np.cos(x[:, 3]) * x[:, 8] ** 2
        - np.sin(x[:, 2] + x[:, 3]) * x[:, 12]
        + np.cos(x[:, 2] + x[:, 3]) * x[:, 13]
    )
    y[:, 5] = x[:, 8] + x[:, 9]

    # right femur
    y[:, 6] = imus[2] * x[:, 16] + g * np.sin(x[:, 4]) + np.sin(x[:, 4]) * x[:, 13] + np.cos(x[:, 4]) * x[:, 12]
    y[:, 7] = imus[2] * x[:, 10] ** 2 + g * np.cos(x[:, 4]) - np.sin(x[:, 4]) * x[:, 12] + np.cos(x[:, 4]) * x[:, 13]
    y[:, 8] = x[:, 10]

    # right fibula
    y[:, 9] = (
        imus[3] * x[:, 16]
        + imus[3] * x[:, 17]
        + g * np.sin(x[:, 4] + x[:, 5])
        + legs[2] * np.sin(x[:, 5]) * x[:, 10] ** 2
        + legs[2] * np.cos(x[:, 5]) * x[:, 16]
        + np.sin(x[:, 4] + x[:, 5]) * x[:, 13]
        + np.cos(x[:, 4] + x[:, 5]) * x[:, 12]
    )
    y[:, 10] = (
        imus[3] * x[:, 10] ** 2
        + 2 * imus[3] * x[:, 10] * x[:, 11]
        + imus[3] * x[:, 11] ** 2
        + g * np.cos(x[:, 4] + x[:, 5])
        - legs[2] * np.sin(x[:, 5]) * x[:, 16]
        + legs[2] * np.cos(x[:, 5]) * x[:, 10] ** 2
        - np.sin(x[:, 4] + x[:, 5]) * x[:, 12]
        + np.cos(x[:, 4] + x[:, 5]) * x[:, 13]
    )
    y[:, 11] = x[:, 10] + x[:, 11]

    # left heel
    y[:, 12] = (
        legs[0] * np.cos(x[:, 2]) * x[:, 8] + legs[1] * (x[:, 8] + x[:, 9]) * np.cos(x[:, 2] + x[:, 3]) + x[:, 6]
    )
    y[:, 13] = (
        legs[0] * np.sin(x[:, 2]) * x[:, 8] + legs[1] * (x[:, 8] + x[:, 9]) * np.sin(x[:, 2] + x[:, 3]) + x[:, 7]
    )
    y[:, 14] = (
        -legs[0] * np.sin(x[:, 2]) * x[:, 8] ** 2
        + legs[0] * np.cos(x[:, 2]) * x[:, 14]
        - legs[1] * (x[:, 8] + x[:, 9]) ** 2 * np.sin(x[:, 2] + x[:, 3])
        + legs[1] * (x[:, 14] + x[:, 15]) * np.cos(x[:, 2] + x[:, 3])
        + x[:, 12]
    )
    y[:, 15] = (
        legs[0] * np.sin(x[:, 2]) * x[:, 14]
        + legs[0] * np.cos(x[:, 2]) * x[:, 8] ** 2
        + legs[1] * (x[:, 8] + x[:, 9]) ** 2 * np.cos(x[:, 2] + x[:, 3])
        + legs[1] * (x[:, 14] + x[:, 15]) * np.sin(x[:, 2] + x[:, 3])
        + x[:, 13]
    )

    # right heel
    y[:, 16] = (
        legs[2] * np.cos(x[:, 4]) * x[:, 10] + legs[3] * (x[:, 10] + x[:, 11]) * np.cos(x[:, 4] + x[:, 5]) + x[:, 6]
    )
    y[:, 17] = (
        legs[2] * np.sin(x[:, 4]) * x[:, 10] + legs[3] * (x[:, 10] + x[:, 11]) * np.sin(x[:, 4] + x[:, 5]) + x[:, 7]
    )
    y[:, 18] = (
        -legs[2] * np.sin(x[:, 4]) * x[:, 10] ** 2
        + legs[2] * np.cos(x[:, 4]) * x[:, 16]
        - legs[3] * (x[:, 10] + x[:, 11]) ** 2 * np.sin(x[:, 4] + x[:, 5])
        + legs[3] * (x[:, 16] + x[:, 17]) * np.cos(x[:, 4] + x[:, 5])
        + x[:, 12]
    )
    y[:, 19] = (
        legs[2] * np.sin(x[:, 4]) * x[:, 16]
        + legs[2] * np.cos(x[:, 4]) * x[:, 10] ** 2
        + legs[3] * (x[:, 10] + x[:, 11]) ** 2 * np.cos(x[:, 4] + x[:, 5])
        + legs[3] * (x[:, 16] + x[:, 17]) * np.sin(x[:, 4] + x[:, 5])
        + x[:, 13]
    )

    y = np.matmul(R, y.T).T
    return y


def compute_jacobian_obs(x, dim_states, dim_observations, g, legs, imus, R):
    nb_particles, _ = x.shape
    df = np.zeros((nb_particles, dim_observations, dim_states))
    # left femur
    df[:, 0, 2] = -x[:, 12] * np.sin(x[:, 2]) + (x[:, 13] + g) * np.cos(x[:, 2])
    df[:, 0, 12] = np.cos(x[:, 2])
    df[:, 0, 13] = np.sin(x[:, 2])
    df[:, 0, 14] = imus[0]
    df[:, 1, 2] = -x[:, 12] * np.cos(x[:, 2]) - (x[:, 13] + g) * np.sin(x[:, 2])
    df[:, 1, 8] = 2 * x[:, 8] * imus[0]
    df[:, 1, 12] = -np.sin(x[:, 2])
    df[:, 1, 13] = np.cos(x[:, 2])
    df[:, 2, 8] = 1

    # left fibula
    df[:, 3, 2] = (
        -x[:, 12] * np.sin(x[:, 2] + x[:, 3]) + x[:, 13] * np.cos(x[:, 2] + x[:, 3]) + g * np.cos(x[:, 2] + x[:, 3])
    )
    df[:, 3, 3] = (
        -x[:, 14] * legs[0] * np.sin(x[:, 3])
        - x[:, 12] * np.sin(x[:, 2] + x[:, 3])
        + x[:, 13] * np.cos(x[:, 2] + x[:, 3])
        + legs[0] * x[:, 8] ** 2 * np.cos(x[:, 3])
        + g * np.cos(x[:, 2] + x[:, 3])
    )
    df[:, 3, 8] = 2 * legs[0] * x[:, 8] * np.sin(x[:, 3])
    df[:, 3, 12] = np.cos(x[:, 2] + x[:, 3])
    df[:, 3, 13] = np.sin(x[:, 2] + x[:, 3])
    df[:, 3, 14] = imus[1] + legs[0] * np.cos(x[:, 3])
    df[:, 3, 15] = imus[1]
    df[:, 4, 2] = (
        -x[:, 12] * np.cos(x[:, 2] + x[:, 3]) - x[:, 13] * np.sin(x[:, 2] + x[:, 3]) - g * np.sin(x[:, 2] + x[:, 3])
    )
    df[:, 4, 3] = (
        -x[:, 14] * legs[0] * np.cos(x[:, 3])
        - x[:, 12] * np.cos(x[:, 2] + x[:, 3])
        - x[:, 13] * np.sin(x[:, 2] + x[:, 3])
        - legs[0] * x[:, 8] ** 2 * np.sin(x[:, 3])
        - g * np.sin(x[:, 2] + x[:, 3])
    )
    df[:, 4, 8] = 2 * legs[0] * x[:, 8] * np.cos(x[:, 3]) + 2 * imus[1] * (x[:, 8] + x[:, 9])
    df[:, 4, 9] = 2 * imus[1] * (x[:, 8] + x[:, 9])
    df[:, 4, 12] = -np.sin(x[:, 2] + x[:, 3])
    df[:, 4, 13] = np.cos(x[:, 2] + x[:, 3])
    df[:, 4, 14] = -legs[0] * np.sin(x[:, 3])
    df[:, 5, 8] = 1
    df[:, 5, 9] = 1

    # right femur
    df[:, 6, 4] = -x[:, 12] * np.sin(x[:, 4]) + (x[:, 13] + g) * np.cos(x[:, 4])
    df[:, 6, 12] = np.cos(x[:, 4])
    df[:, 6, 13] = np.sin(x[:, 4])
    df[:, 6, 16] = imus[2]
    df[:, 7, 4] = -x[:, 12] * np.cos(x[:, 4]) - (x[:, 13] + g) * np.sin(x[:, 4])
    df[:, 7, 10] = 2 * x[:, 10] * imus[2]
    df[:, 7, 12] = -np.sin(x[:, 4])
    df[:, 7, 13] = np.cos(x[:, 4])
    df[:, 8, 10] = 1

    # right fibula
    df[:, 9, 4] = (
        -x[:, 12] * np.sin(x[:, 4] + x[:, 5]) + x[:, 13] * np.cos(x[:, 4] + x[:, 5]) + g * np.cos(x[:, 4] + x[:, 5])
    )
    df[:, 9, 5] = (
        -x[:, 16] * legs[2] * np.sin(x[:, 5])
        - x[:, 12] * np.sin(x[:, 4] + x[:, 5])
        + x[:, 13] * np.cos(x[:, 4] + x[:, 5])
        + legs[2] * x[:, 10] ** 2 * np.cos(x[:, 5])
        + g * np.cos(x[:, 4] + x[:, 5])
    )
    df[:, 9, 10] = 2 * legs[2] * x[:, 10] * np.sin(x[:, 5])
    df[:, 9, 12] = np.cos(x[:, 4] + x[:, 5])
    df[:, 9, 13] = np.sin(x[:, 4] + x[:, 5])
    df[:, 9, 16] = imus[3] + legs[2] * np.cos(x[:, 5])
    df[:, 9, 17] = imus[3]
    df[:, 10, 4] = (
        -x[:, 12] * np.cos(x[:, 4] + x[:, 5]) - x[:, 13] * np.sin(x[:, 4] + x[:, 5]) - g * np.sin(x[:, 4] + x[:, 5])
    )
    df[:, 10, 5] = (
        -x[:, 16] * legs[2] * np.cos(x[:, 5])
        - x[:, 12] * np.cos(x[:, 4] + x[:, 5])
        - x[:, 13] * np.sin(x[:, 4] + x[:, 5])
        - legs[2] * x[:, 10] ** 2 * np.sin(x[:, 5])
        - g * np.sin(x[:, 4] + x[:, 5])
    )
    df[:, 10, 10] = 2 * legs[2] * x[:, 10] * np.cos(x[:, 5]) + 2 * imus[3] * (x[:, 10] + x[:, 11])
    df[:, 10, 11] = 2 * imus[3] * (x[:, 10] + x[:, 11])
    df[:, 10, 12] = -np.sin(x[:, 4] + x[:, 5])
    df[:, 10, 13] = np.cos(x[:, 4] + x[:, 5])
    df[:, 10, 16] = -legs[2] * np.sin(x[:, 5])
    df[:, 11, 10] = 1
    df[:, 11, 11] = 1

    # left heel
    df[:, 12, 2] = -x[:, 8] * legs[0] + np.sin(x[:, 2]) - legs[1] * (x[:, 8] + x[:, 9]) * np.sin(x[:, 2] + x[:, 3])
    df[:, 12, 3] = -legs[1] * (x[:, 8] + x[:, 9]) * np.sin(x[:, 2] + x[:, 3])
    df[:, 12, 6] = 1
    df[:, 12, 8] = legs[0] * np.cos(x[:, 2]) + legs[1] * np.cos(x[:, 2] + x[:, 3])
    df[:, 12, 9] = legs[1] * np.cos(x[:, 2] + x[:, 3])
    df[:, 13, 2] = x[:, 8] * legs[0] * np.cos(x[:, 2]) + legs[1] * (x[:, 8] + x[:, 9]) * np.cos(x[:, 2] + x[:, 3])
    df[:, 13, 3] = legs[1] * (x[:, 8] + x[:, 9]) * np.cos(x[:, 2] + x[:, 3])
    df[:, 13, 7] = 1
    df[:, 13, 8] = legs[0] * np.sin(x[:, 2]) + legs[1] * np.sin(x[:, 2] + x[:, 3])
    df[:, 13, 9] = legs[1] * np.sin(x[:, 2] + x[:, 3])
    df[:, 14, 2] = (
        -legs[0] * (x[:, 14] * np.sin(x[:, 2]) + x[:, 8] ** 2 * np.cos(x[:, 2]))
        - legs[1] * (x[:, 14] + x[:, 15]) * np.sin(x[:, 2] + x[:, 3])
        - legs[1] * (x[:, 8] + x[:, 9]) ** 2 * np.cos(x[:, 2] + x[:, 3])
    )
    df[:, 14, 3] = -legs[1] * (x[:, 14] + x[:, 15]) * np.sin(x[:, 2] + x[:, 3]) - legs[1] * (
        x[:, 8] + x[:, 9]
    ) ** 2 * np.cos(x[:, 2] + x[:, 3])
    df[:, 14, 8] = -2 * legs[0] * x[:, 8] * np.sin(x[:, 2]) - 2 * legs[1] * (x[:, 8] + x[:, 9]) * np.sin(
        x[:, 2] + x[:, 3]
    )
    df[:, 14, 9] = -2 * legs[1] * (x[:, 8] + x[:, 9]) * np.sin(x[:, 2] + x[:, 3])
    df[:, 14, 12] = 1
    df[:, 14, 14] = legs[0] * np.cos(x[:, 2]) + legs[1] * np.cos(x[:, 2] + x[:, 3])
    df[:, 14, 15] = legs[1] * np.cos(x[:, 2] + x[:, 3])
    df[:, 15, 2] = (
        -legs[0] * (-x[:, 14] * np.cos(x[:, 2]) + x[:, 8] ** 2 * np.sin(x[:, 2]))
        + legs[1] * (x[:, 14] + x[:, 15]) * np.cos(x[:, 2] + x[:, 3])
        - legs[1] * (x[:, 8] + x[:, 9]) ** 2 * np.sin(x[:, 2] + x[:, 3])
    )
    df[:, 15, 3] = legs[1] * (x[:, 14] + x[:, 15]) * np.cos(x[:, 2] + x[:, 3]) - legs[1] * (
        x[:, 8] + x[:, 9]
    ) ** 2 * np.sin(x[:, 2] + x[:, 3])
    df[:, 15, 8] = 2 * legs[0] * x[:, 8] * np.cos(x[:, 2]) + 2 * legs[1] * (x[:, 8] + x[:, 9]) * np.cos(
        x[:, 2] + x[:, 3]
    )
    df[:, 15, 9] = 2 * legs[1] * (x[:, 8] + x[:, 9]) * np.cos(x[:, 2] + x[:, 3])
    df[:, 15, 13] = 1
    df[:, 15, 14] = legs[0] * np.sin(x[:, 2]) + legs[1] * np.sin(x[:, 2] + x[:, 3])
    df[:, 15, 15] = legs[1] * np.sin(x[:, 2] + x[:, 3])

    # right heel
    df[:, 16, 4] = -x[:, 10] * legs[2] + np.sin(x[:, 4]) - legs[3] * (x[:, 10] + x[:, 11]) * np.sin(x[:, 4] + x[:, 5])
    df[:, 16, 5] = -legs[3] * (x[:, 10] + x[:, 11]) * np.sin(x[:, 4] + x[:, 5])
    df[:, 16, 6] = 1
    df[:, 16, 10] = legs[2] * np.cos(x[:, 4]) + legs[3] * np.cos(x[:, 4] + x[:, 5])
    df[:, 16, 11] = legs[3] * np.cos(x[:, 4] + x[:, 5])
    df[:, 17, 4] = x[:, 10] * legs[2] * np.cos(x[:, 4]) + legs[3] * (x[:, 10] + x[:, 11]) * np.cos(x[:, 4] + x[:, 5])
    df[:, 17, 5] = legs[3] * (x[:, 10] + x[:, 11]) * np.cos(x[:, 4] + x[:, 5])
    df[:, 17, 7] = 1
    df[:, 17, 10] = legs[2] * np.sin(x[:, 4]) + legs[3] * np.sin(x[:, 4] + x[:, 5])
    df[:, 17, 11] = legs[3] * np.sin(x[:, 4] + x[:, 5])
    df[:, 18, 4] = (
        -legs[2] * (x[:, 16] * np.sin(x[:, 4]) + x[:, 10] ** 2 * np.cos(x[:, 4]))
        - legs[3] * (x[:, 16] + x[:, 17]) * np.sin(x[:, 4] + x[:, 5])
        - legs[3] * (x[:, 10] + x[:, 11]) ** 2 * np.cos(x[:, 4] + x[:, 5])
    )
    df[:, 18, 5] = -legs[3] * (x[:, 16] + x[:, 17]) * np.sin(x[:, 4] + x[:, 5]) - legs[3] * (
        x[:, 10] + x[:, 11]
    ) ** 2 * np.cos(x[:, 4] + x[:, 5])
    df[:, 18, 10] = -2 * legs[2] * x[:, 10] * np.sin(x[:, 4]) - 2 * legs[3] * (x[:, 10] + x[:, 11]) * np.sin(
        x[:, 4] + x[:, 5]
    )
    df[:, 18, 11] = -2 * legs[3] * (x[:, 10] + x[:, 11]) * np.sin(x[:, 4] + x[:, 5])
    df[:, 18, 12] = 1
    df[:, 18, 16] = legs[2] * np.cos(x[:, 4]) + legs[3] * np.cos(x[:, 4] + x[:, 5])
    df[:, 18, 17] = legs[3] * np.cos(x[:, 4] + x[:, 5])
    df[:, 19, 4] = (
        -legs[2] * (-x[:, 16] * np.cos(x[:, 4]) + x[:, 10] ** 2 * np.sin(x[:, 4]))
        + legs[3] * (x[:, 16] + x[:, 17]) * np.cos(x[:, 4] + x[:, 5])
        - legs[3] * (x[:, 10] + x[:, 11]) ** 2 * np.sin(x[:, 4] + x[:, 5])
    )
    df[:, 19, 5] = legs[3] * (x[:, 16] + x[:, 17]) * np.cos(x[:, 4] + x[:, 5]) - legs[3] * (
        x[:, 10] + x[:, 11]
    ) ** 2 * np.sin(x[:, 4] + x[:, 5])
    df[:, 19, 10] = 2 * legs[2] * x[:, 10] * np.cos(x[:, 4]) + 2 * legs[3] * (x[:, 10] + x[:, 11]) * np.cos(
        x[:, 4] + x[:, 5]
    )
    df[:, 19, 11] = 2 * legs[3] * (x[:, 10] + x[:, 11]) * np.cos(x[:, 4] + x[:, 5])
    df[:, 19, 13] = 1
    df[:, 19, 16] = legs[2] * np.sin(x[:, 4]) + legs[3] * np.sin(x[:, 4] + x[:, 5])
    df[:, 19, 17] = legs[3] * np.sin(x[:, 4] + x[:, 5])

    df = np.matmul(R, df)
    return df


def state_to_obs_linear(x, xp, dim_states, dim_observations, g, legs, imus, R):
    if xp is None:
        xp = np.zeros(x.shape)
    nb_steps, _ = x.shape
    df = compute_jacobian_obs(xp, dim_states, dim_observations, g, legs, imus, R)
    y = state_to_obs(xp, dim_observations, g, legs, imus, R) + np.einsum("ijk, ik -> ij", df, x - xp)
    return y

In [7]:
class TwoLegModel(ssm.StateSpaceModel):
    def __init__(
        self,
        dt=0.01,
        dim_states=18,
        dim_observations=20,
        femur_left=0.5,
        fibula_left=0.6,
        femur_right=0.5,
        fibula_right=0.6,
        pos_imu0=0.34,
        pos_imu1=0.29,
        pos_imu2=0.315,
        pos_imu3=0.33,
        alpha0=0.0,
        alpha1=0.0,
        alpha2=0.0,
        alpha3=0.0,
        b0=np.array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
        factor_Q0=0.1,
        lambda_x=10000.0,
        lambda_y=1000.0,
        lambda_phi=10000000.0,
        sigma_imu_acc=0.1,
        sigma_imu_gyro=0.1,
        sigma_press_velo=0.1,
        sigma_press_acc=1.0,
        factor_proposal=1.2,
    ):
        ssm.StateSpaceModel().__init__()
        self.dt = dt
        self.dim_states = dim_states
        self.dim_observations = dim_observations
        self.A = np.zeros((dim_states, dim_states))
        self.set_process_transition_matrix()
        self.g = 9.81
        self.femur_left = femur_left
        self.fibula_left = fibula_left
        self.femur_right = femur_right
        self.fibula_right = fibula_right
        self.len_legs = np.array([self.femur_left, self.fibula_left, self.femur_right, self.fibula_right])
        self.pos_imu0 = pos_imu0
        self.pos_imu1 = pos_imu1
        self.pos_imu2 = pos_imu2
        self.pos_imu3 = pos_imu3
        self.pos_imus = np.array([self.pos_imu0, self.pos_imu1, self.pos_imu2, self.pos_imu3])
        self.alpha0 = alpha0
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.alpha3 = alpha3
        self.R = np.eye(self.dim_observations)
        self.set_imu_rotation_matrices()
        self.b0 = b0
        self.factor_Q0 = factor_Q0
        self.Q0 = self.factor_Q0 * np.eye(self.dim_states)
        self.lambda_x = lambda_x
        self.lambda_y = lambda_y
        self.lambda_phi = lambda_phi
        self.sigma_imu_acc = sigma_imu_acc
        self.sigma_imu_gyro = sigma_imu_gyro
        self.sigma_press_velo = sigma_press_velo
        self.sigma_press_acc = sigma_press_acc
        self.Q = np.zeros((self.dim_states, self.dim_states))
        self.set_process_covariance()
        self.V = np.zeros((self.dim_observations, self.dim_observations))
        self.set_observation_covariance()
        self.factor_proposal = factor_proposal

    def set_process_transition_matrix(self):
        self.A = np.eye(self.dim_states)
        for row in range(0, self.dim_states):
            for col in range(0, self.dim_states):
                if row + 6 == col:
                    self.A[row, col] = self.dt
                if row + 12 == col:
                    self.A[row, col] = self.dt**2 / 2.0

    def set_process_covariance(self):
        block_size = self.dim_states // 3
        for row in range(0, self.dim_states):
            for col in range(0, self.dim_states):
                if row < block_size:
                    if row == col:
                        self.Q[row, col] = self.dt**5 / 20.0
                    elif row + 6 == col:
                        self.Q[row, col] = self.dt**4 / 8.0
                        self.Q[col, row] = self.dt**4 / 8.0
                    elif row + 12 == col:
                        self.Q[row, col] = self.dt**3 / 6.0
                        self.Q[col, row] = self.dt**3 / 6.0
                elif block_size <= row < 2 * block_size:
                    if row == col:
                        self.Q[row, col] = self.dt**3 / 3.0
                    elif row + 6 == col:
                        self.Q[row, col] = self.dt**2 / 2.0
                        self.Q[col, row] = self.dt**2 / 2.0
                elif 2 * block_size <= row:
                    if row == col:
                        self.Q[row, col] = self.dt
        idx_groups = [[0, 6, 12], [1, 7, 13], [2, 8, 14], [3, 9, 15], [4, 10, 16], [5, 11, 17]]
        scale_factors = [
            self.lambda_x,
            self.lambda_y,
            self.lambda_phi,
            self.lambda_phi,
            self.lambda_phi,
            self.lambda_phi,
        ]
        for factor, idxs in zip(scale_factors, idx_groups):
            for row, col in itertools.product(idxs, idxs):
                self.Q[row, col] *= factor

    def set_observation_covariance(self):
        self.V = np.diag(
            [
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_imu_acc,
                self.sigma_imu_acc,
                self.sigma_imu_gyro,
                self.sigma_press_velo,
                self.sigma_press_velo,
                self.sigma_press_acc,
                self.sigma_press_acc,
                self.sigma_press_velo,
                self.sigma_press_velo,
                self.sigma_press_acc,
                self.sigma_press_acc,
            ]
        )

    def set_imu_rotation_matrices(self):
        R0 = create_rotation_matrix_z(self.alpha0)
        R1 = create_rotation_matrix_z(self.alpha1)
        R2 = create_rotation_matrix_z(self.alpha2)
        R3 = create_rotation_matrix_z(self.alpha3)
        # R = block_diag(R0, R0, R1, R1, R2, R2, R3, R3, np.eye(12))
        R = block_diag(R0, R1, R2, R3, np.eye(8))
        self.R = R

    def state_transition(self, xp):
        return np.matmul(self.A, xp.T).T

    def state_to_observation(self, x):
        return state_to_obs(x, self.dim_observations, self.g, self.len_legs, self.pos_imus, self.R)

    def PX0(self):
        return dists.MvNormal(loc=self.b0, cov=self.Q0)

    def PX(self, t, xp):
        return dists.MvNormal(loc=self.state_transition(xp), cov=self.Q)

    def PY(self, t, xp, x):
        return dists.MvNormal(loc=self.state_to_observation(x), cov=self.V)

    def compute_observation_derivatives(self, x):
        return compute_jacobian_obs(
            x, self.dim_states, self.dim_observations, self.g, self.len_legs, self.pos_imus, self.R
        )

    def compute_ekf_proposal(self, xp, data_t):
        mask_not_nan = np.invert(np.isnan(data_t))
        mask_2d = np.outer(mask_not_nan, mask_not_nan)
        nb_non_nan = np.sum(mask_not_nan)

        # compute masked covariance and Jacobian
        x_hat = self.state_transition(xp)
        dh = self.compute_observation_derivatives(x_hat)
        dh = dh[:, mask_not_nan.flatten(), :]
        V_masked = np.reshape(self.V[mask_2d], (nb_non_nan, nb_non_nan))

        # EKF
        dh_Q = np.matmul(dh, self.Q)
        S_inv = np.linalg.inv(np.matmul(dh_Q, np.transpose(dh, (0, 2, 1))) + V_masked)
        kalman_gain = np.matmul(np.transpose(dh_Q, (0, 2, 1)), S_inv)
        prediction_err = data_t[mask_not_nan] - self.state_to_observation(x_hat)[:, mask_not_nan.flatten()]
        mu = x_hat + np.einsum("ijk, ik -> ij", kalman_gain, prediction_err)
        Sigma = self.Q - np.matmul(kalman_gain, dh_Q)

        return mu, Sigma

    def proposal0(self, data):
        return self.PX0()

    def proposal(self, t, xp, data):
        mu, Sigma = self.compute_ekf_proposal(xp, data[t])
        Sigma = self.factor_proposal * np.mean(Sigma, axis=0)
        return dists.MvNormal(loc=mu, cov=Sigma)

In [8]:
model = TwoLegModel()

In [13]:
nb_particles = 200
nb_timesteps = observations.shape[0]
ESSrmin = 0.5

fk_model = ssm.Bootstrap(ssm=model, data=observations.to_numpy())  # bootstrap filter
# fk_model = ssm.GuidedPF(ssm=model, data=observations.to_numpy())  # guided filter

pf = particles.SMC(
    fk=fk_model,
    N=nb_particles,
    ESSrmin=ESSrmin,
    store_history=True,
    collect=[Moments()],
)
start_user, start_process = time.time(), time.process_time()
pf.run()
end_user, end_process = time.time(), time.process_time()
s_user = end_user - start_user
s_process = end_process - start_process
print(
    f"Time user {s_user // 60:.0f}min {s_user % 60:.0f}s; time processor {s_process // 60:.0f}min {s_process % 60:.0f}s"
)
print(f"Resampled {np.sum(pf.summaries.rs_flags)} of totally {nb_timesteps} steps.")
loglikelihood = pf.summaries.logLts[-1]
print(f"Log likelihood = {loglikelihood:.3E}")

Time user 0min 2s; time processor 0min 14s
Resampled 999 of totally 1000 steps.
Log likelihood = -2.837E+08
